In [1]:
import argparse
import os
import sys
from pathlib import Path
from IPython.display import clear_output, display
import ipywidgets

import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn

# FILE = Path('./')
ROOT = Path('/home/rodrigo/Documents/usp/tcc/yolov5/')  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

In [70]:
class RoadFollower:
    def __init__(self, x_last=0, y_last=0, threshold=100):
        self.x_last = x_last
        self.y_last = y_last
        self.setpoint = (0.5, 0.5)
        self.threshold = threshold
        self.angle = 0.0
        self.angle_last = 0.0
        self.start_sliders()
        # self.start_robot()

    # def start_robot(self):
    #     self.robot = Robot()

    def start_sliders(self):
        self.speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
        self.steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
        self.steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.25, description='steering kd')
        self.steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

        self.x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
        self.y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
        self.steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
        self.speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

        self.left_motor = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='left motor')
        self.right_motor = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='right motor')

    def display_control_sliders(self):
        display(
            self.speed_gain_slider,
            self.steering_gain_slider,
            self.steering_dgain_slider,
            self.steering_bias_slider
        )

    def display_monitor_sliders(self):
        display(self.y_slider, self.speed_slider)
        display(self.x_slider, self.steering_slider)
        display(self.left_motor, self.right_motor)

    def execute(self, image):
        xy, image = self.update(image)
        # image = change['new']
        # xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0

        self.x_slider.value = x
        self.y_slider.value = y

        self.speed_slider.value = self.speed_gain_slider.value

        self.angle = np.arctan2(x, y)
        pid = self.angle * self.steering_gain_slider.value\
              + (self.angle - self.angle_last) * self.steering_dgain_slider.value
        self.angle_last = self.angle

        self.steering_slider.value = pid + self.steering_bias_slider.value

        # robot.left_motor.value = 
        self.left_motor.value = max(
            min(self.speed_slider.value + self.steering_slider.value, 1.0),
            0.0
        )
        self.right_motor.value = max(
            min(self.speed_slider.value - self.steering_slider.value, 1.0),
            0.0
        )
        return image

    def update(self, image):
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        roi = [[250,-1],[0,-1]]
        image = image[roi[0][0]:roi[0][1], roi[1][0]:roi[1][1]]
        # image = image[300:415, :]  # Region Of Interest
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # print(image.shape)
        (thresh, image) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        # Blackline = cv2.inRange(image, (0,0,0), (self.threshold,self.threshold,self.threshold))
        Blackline = cv2.inRange(image, 0, thresh)
        kernel = np.ones((3,3), np.uint8)
        Blackline = cv2.erode(Blackline, kernel, iterations=5)
        Blackline = cv2.dilate(Blackline, kernel, iterations=9)	
        contours_blk, hierarchy_blk = cv2.findContours(Blackline.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        contours_blk_len = len(contours_blk)
        if contours_blk_len > 0 :
            if contours_blk_len == 1 :
                blackbox = cv2.minAreaRect(contours_blk[0])
            else:
                canditates=[]
                off_bottom = 0	   
                for con_num in range(contours_blk_len):		
                    blackbox = cv2.minAreaRect(contours_blk[con_num])
                    (x_min, y_min), (w_min, h_min), ang = blackbox		
                    box = cv2.boxPoints(blackbox)
                    (x_box,y_box) = box[0]
                    if y_box > 358 :		 
                        off_bottom += 1
                    canditates.append((y_box,con_num,x_min,y_min))		
                canditates = sorted(canditates)
                if off_bottom > 1:	    
                    canditates_off_bottom=[]
                    for con_num in range ((contours_blk_len - off_bottom), contours_blk_len):
                        (y_highest,con_highest,x_min, y_min) = canditates[con_num]		
                        total_distance = (abs(x_min - self.x_last)**2 + abs(y_min - self.y_last)**2)**0.5
                        canditates_off_bottom.append((total_distance,con_highest))
                    canditates_off_bottom = sorted(canditates_off_bottom)         
                    (total_distance,con_highest) = canditates_off_bottom[0]         
                    blackbox = cv2.minAreaRect(contours_blk[con_highest])	   
                else:		
                    (y_highest,con_highest,x_min, y_min) = canditates[contours_blk_len-1]
                    blackbox = cv2.minAreaRect(contours_blk[con_highest])
            (x_min, y_min), (w_min, h_min), ang = blackbox
            self.x_last = x_min
            self.y_last = y_min
            box = cv2.boxPoints(blackbox)
            box = np.int0(box)
            cv2.drawContours(image,[box],0,(0,0,0),3)
            cv2.line(image, (int(x_min),0 ), (int(x_min),50 ), (0,0,0),3)
            cv2.circle(image, (image.shape[1]//2, image.shape[0]//2), 5, (0,0,0), 3)
            cv2.circle(image, (int(x_min), int(y_min)), 5, (255,255,255), 3)
            xy = [(x_min - image.shape[1]/2.0) / image.shape[1], (y_min - image.shape[0]/2.0) / image.shape[0]]
        else:
            xy = [0.0, 0.0]
            # error = 0
        return xy, image
        # return image, error

In [3]:
# Parameters
weights = ROOT / 'best.pt'  # model.pt path(s)
source = 0  # file/dir/URL/glob, 0 for webcam
imgsz = [416]  # inference size (pixels)
conf_thres = 0.25  # confidence threshold
iou_thres = 0.45  # NMS IOU threshold
max_det = 1000  # maximum detections per image
device = ''  # cuda device, i.e. 0 or 0,1,2,3 or cpu
view_img = False  # show results
save_txt = False  # save results to *.txt
save_conf = False  # save confidences in --save-txt labels
save_crop = False  # save cropped prediction boxes
nosave = False  # do not save images/videos
classes = None  # filter by class: --class 0, or --class 0 2 3
agnostic_nms = False  # class-agnostic NMS
augment = False  # augmented inference
visualize = False  # visualize features
update = False  # update all models
project = ROOT / 'runs/detect'  # save results to project/name
name = 'exp'  # save results to project/name
exist_ok = False  # existing project/name ok, do not increment
line_thickness = 3  # bounding box thickness (pixels)
hide_labels = False  # hide labels
hide_conf = False  # hide confidences
half = False  # use FP16 half-precision inference
dnn = False  # use OpenCV DNN for ONNX inference

In [4]:
imgsz *= 2 if len(imgsz) == 1 else 1  # expand

In [5]:
imgsz

[416, 416]

In [6]:
source = str(source)
save_img = not nosave and not source.endswith('.txt')  # save inference images
is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
if is_url and is_file:
    source = check_file(source)  # download

In [7]:
# Directories
save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

In [8]:
# Load model
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn)
stride, names, pt, jit, onnx = model.stride, model.names, model.pt, model.jit, model.onnx
imgsz = check_img_size(imgsz, s=stride)  # check image size

YOLOv5 🚀 v6.0-83-gcc91ae5 torch 1.10.0+cu102 CUDA:0 (NVIDIA GeForce MX150, 4042MiB)

Fusing layers... 
Model Summary: 213 layers, 7031701 parameters, 0 gradients, 15.9 GFLOPs


In [9]:
imgsz

[416, 416]

In [10]:
stride

32

In [11]:
# Dataloader
view_img = check_imshow()
cudnn.benchmark = True  # set True to speed up constant image size inference
dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt and not jit)
bs = len(dataset)  # batch_size

1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)



In [12]:
vid_path, vid_writer = [None] * bs, [None] * bs

In [13]:
if pt and device.type != 'cpu':
    model(torch.zeros(1, 3, *imgsz).to(device).type_as(next(model.model.parameters())))  # warmup
dt, seen = [0.0, 0.0, 0.0], 0

In [75]:
road_follower = RoadFollower()

In [76]:
road_follower.display_control_sliders()

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.25, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [77]:
road_follower.display_monitor_sliders()

FloatSlider(value=0.0, description='y', max=1.0, orientation='vertical')

FloatSlider(value=0.0, description='speed', max=1.0, orientation='vertical')

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='left motor', max=1.0, orientation='vertical')

FloatSlider(value=0.0, description='right motor', max=1.0, orientation='vertical')

In [78]:
try:
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        # visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=False)
        t3 = time_sync()
        dt[1] += t3 - t2
        # print("="*50)
        # print("PRED FIRST:\n", pred)

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        # print("PRED NMS:\n", pred)
        # print("="*50)
        dt[2] += time_sync() - t3
        im0 = im0s.copy()
        det = pred[0]
        for c in det[:, -1].unique():
            n = (det[:, -1] == c).sum()  # detections per class
            s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
        # image = np.asarray(im.cpu())
        image = im[0].cpu().permute(1, 2, 0).numpy() * 255
        image = np.asarray(image).astype('uint8').copy()
        # image, _ = road_follower.update(image)
        image = road_follower.execute(image)
        cv2.putText(image,str(s),(10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        cv2.imshow("Test", image)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
except:
    cv2.destroyAllWindows()

In [69]:
cv2.destroyAllWindows()